In [1]:
from math import sqrt
import pandas as pd
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.models import Sequential
from keras.layers import Dense, LSTM
from numpy import concatenate

Using TensorFlow backend.
/Users/alexellard/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/alexellard/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/alexellard/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/alexellard/opt/anacond

In [2]:
df = pd.read_csv('/Users/alexellard/Downloads/scenarios_data.csv', header=0, index_col=0)
df

,reservoir_demand,node_1_demand,node_2_demand,node_3_demand,node_4_demand,node_5_demand,node_6_demand,node_7_demand,node_8_demand,node_9_demand,...,link_23_flow,link_24_flow,link_25_flow,link_26_flow,link_27_flow,link_28_flow,link_29_flow,link_30_flow,link_31_flow,link_32_flow
Timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,-3348.0,154.8,169.2,18.0,118.8,169.2,234.0,86.4,82.8,104.4,...,439.2,-205.2,-75.6,-18.0,118.8,68.4,7.2,-54.0,75.6,212.4
2017-01-01 00:30:00,-2959.2,136.8,147.6,18.0,108.0,144.0,237.6,79.2,75.6,90.0,...,392.4,-187.2,-72.0,-21.6,108.0,61.2,3.6,-54.0,72.0,183.6
2017-01-01 01:00:00,-2692.8,129.6,126.0,14.4,104.4,133.2,205.2,72.0,72.0,90.0,...,349.2,-176.4,-64.8,-18.0,93.6,50.4,3.6,-39.6,54.0,151.2
2017-01-01 01:30:00,-2379.6,104.4,100.8,10.8,86.4,122.4,180.0,68.4,57.6,82.8,...,316.8,-158.4,-54.0,-10.8,79.2,46.8,7.2,-32.4,50.4,140.4
2017-01-01 02:00:00,-2228.4,104.4,104.4,10.8,79.2,122.4,183.6,68.4,54.0,75.6,...,277.2,-144.0,-57.6,-18.0,72.0,39.6,7.2,-32.4,43.2,118.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 21:30:00,-5306.4,205.2,208.8,39.6,183.6,284.4,316.8,176.4,133.2,129.6,...,604.8,-259.2,32.4,115.2,169.2,100.8,10.8,-100.8,122.4,309.6
2017-12-31 22:00:00,-4910.4,208.8,183.6,39.6,154.8,252.0,295.2,165.6,133.2,133.2,...,565.2,-241.2,21.6,100.8,154.8,86.4,18.0,-75.6,100.8,288.0
2017-12-31 22:30:00,-4683.6,187.2,183.6,36.0,147.6,230.4,320.4,147.6,118.8,126.0,...,529.2,-226.8,21.6,100.8,147.6,86.4,14.4,-86.4,108.0,266.4


In [3]:
# convert series to supervised learning
def series_to_supervised(data, n_in, n_out, vars_to_predict=None):
    n_vars = data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i-1))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(n_out):
        cols.append(df[vars_to_predict].shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in vars_to_predict]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in vars_to_predict]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    return agg

values = df.values
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

# node 1 indexes
target_variable_demand_node1 = [1]
target_variable_pressures_node1 = [33]
target_variable_flows_node1 = [64]
#node 16 indexes
target_variable_demand_node16 = [16]
target_variable_pressures_node16 = [48]
target_variable_flows_node16 = [79]
#node 29 indexes
target_variable_demand_node29 = [29]
target_variable_pressures_node29 = [61]
target_variable_flows_node29 = [92]


reframed_demand_node1 = series_to_supervised(scaled, 1, 1, vars_to_predict=target_variable_demand_node1)
reframed_pressures_node1 = series_to_supervised(scaled, 1, 1, vars_to_predict=target_variable_pressures_node1)
reframed_flows_node1 = series_to_supervised(scaled, 1, 1, vars_to_predict=target_variable_flows_node1)

reframed_demand_node16 = series_to_supervised(scaled, 1, 1, vars_to_predict=target_variable_demand_node16)
reframed_pressures_node16 = series_to_supervised(scaled, 1, 1, vars_to_predict=target_variable_pressures_node16)
reframed_flows_node16 = series_to_supervised(scaled, 1, 1, vars_to_predict=target_variable_flows_node16)

reframed_demand_node29 = series_to_supervised(scaled, 1, 1, vars_to_predict=target_variable_demand_node29)
reframed_pressures_node29 = series_to_supervised(scaled, 1, 1, vars_to_predict=target_variable_pressures_node29)
reframed_flows_node29 = series_to_supervised(scaled, 1, 1, vars_to_predict=target_variable_flows_node29)

reframed_demand_node1 = reframed_demand_node1.drop(reframed_demand_node1.columns[1], axis=1)
reframed_pressures_node1 = reframed_pressures_node1.drop(reframed_pressures_node1.columns[33], axis=1)
reframed_flows_node1 = reframed_flows_node1.drop(reframed_flows_node1.columns[64], axis=1)

reframed_demand_node16 = reframed_demand_node16.drop(reframed_demand_node16.columns[16], axis=1)
reframed_pressures_node16 = reframed_pressures_node16.drop(reframed_pressures_node16.columns[48], axis=1)
reframed_flows_node16 = reframed_flows_node16.drop(reframed_flows_node16.columns[79], axis=1)

reframed_demand_node29 = reframed_demand_node29.drop(reframed_demand_node29.columns[29], axis=1)
reframed_pressures_node29 = reframed_pressures_node29.drop(reframed_pressures_node29.columns[61], axis=1)
reframed_flows_node29 = reframed_flows_node29.drop(reframed_flows_node29.columns[92], axis=1)

In [4]:
demands = [reframed_demand_node1, reframed_demand_node16, reframed_demand_node29]
pressures = [reframed_pressures_node1, reframed_pressures_node16, reframed_pressures_node29]
flows = [reframed_flows_node1, reframed_flows_node16, reframed_flows_node29]

In [5]:
for node in demands:
    # split into train and test sets
    demand_values = node.values
    train, test = train_test_split(demand_values, test_size=0.2, random_state=0)

    # split into input and outputs
    train_X, train_y = train[:, :-1], train[:, -1]
    test_X, test_y = test[:, :-1], test[:, -1]

    # reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
    print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

    model = Sequential()
    model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam')
    # fit network
    model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
    
    # make predictions on the test data
    y_pred = model.predict(test_X)
    test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))

    test_y = test_y.reshape((len(test_y), 1))
    inv_test_y = concatenate((test_y, test_X[:, -96:]), axis=1)
    test_y_inverse = scaler.inverse_transform(inv_test_y)
    test_y_inverse = test_y_inverse[:,0]

    y_pred = y_pred.reshape((len(y_pred), 1))
    inv_y_pred = concatenate((y_pred, test_X[:, -96:]), axis=1)
    y_pred_inverse = scaler.inverse_transform(inv_y_pred)
    y_pred_inverse = y_pred_inverse[:,0]

    # evaluate scaled errors
    mae = mean_absolute_error(test_y_inverse, y_pred_inverse)
    print('Inverted Mean Absolute Error:', mae)
    rmse = sqrt(mean_squared_error(test_y_inverse, y_pred_inverse))
    print('Inverted Test RMSE: %.3f' % rmse)
    r2 = r2_score(test_y_inverse, y_pred_inverse)
    print('Inverted Test R^2: %.3f' % r2)

(70080, 1, 95) (70080,) (17520, 1, 95) (17520,)

Train on 70080 samples, validate on 17520 samples
Epoch 1/50
 - 6s - loss: 0.0257 - val_loss: 0.0247
Epoch 2/50
 - 5s - loss: 0.0226 - val_loss: 0.0219
Epoch 3/50
 - 5s - loss: 0.0223 - val_loss: 0.0219
Epoch 4/50
 - 5s - loss: 0.0221 - val_loss: 0.0218
Epoch 5/50
 - 5s - loss: 0.0220 - val_loss: 0.0220
Epoch 6/50
 - 5s - loss: 0.0220 - val_loss: 0.0221
Epoch 7/50
 - 5s - loss: 0.0219 - val_loss: 0.0218
Epoch 8/50
 - 5s - loss: 0.0218 - val_loss: 0.0217
Epoch 9/50
 - 5s - loss: 0.0217 - val_loss: 0.0215
Epoch 10/50
 - 7s - loss: 0.0216 - val_loss: 0.0215
Epoch 11/50
 - 7s - loss: 0.0216 - val_loss: 0.0214
Epoch 12/50
 - 7s - loss: 0.0216 - val_loss: 0.0215
Epoch 13/50
 - 7s - loss: 0.0215 - val_loss: 0.0215
Epoch 14/50
 - 7s - loss: 0.0215 - val_loss: 0.0212
Epoch 15/50
 - 7s - loss: 0.0215 - val_loss: 0.0211
Epoch 16/50
 - 7s - loss: 0.0214 - val_loss: 0.0210
Epoch 17/50
 - 7s - loss: 0.0214 - val_loss: 0.0215
Epoch 18/50
 - 7s - loss: 

Epoch 46/50
 - 6s - loss: 0.0062 - val_loss: 0.0053
Epoch 47/50
 - 8s - loss: 0.0064 - val_loss: 0.0063
Epoch 48/50
 - 7s - loss: 0.0062 - val_loss: 0.0062
Epoch 49/50
 - 7s - loss: 0.0063 - val_loss: 0.0053
Epoch 50/50
 - 7s - loss: 0.0062 - val_loss: 0.0058
Inverted Mean Absolute Error: 49.285313
Inverted Test RMSE: 65.698
Inverted Test R^2: 0.998


In [6]:
for node in pressures:
    # split into train and test sets
    pressure_values = node.values
    train, test = train_test_split(pressure_values, test_size=0.2, random_state=0)

    # split into input and outputs
    train_X, train_y = train[:, :-1], train[:, -1]
    test_X, test_y = test[:, :-1], test[:, -1]

    # reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
    print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

    model = Sequential()
    model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam')
    # fit network
    model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=1, shuffle=False)
    
    # make predictions on the test data
    y_pred = model.predict(test_X)
    test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))

    test_y = test_y.reshape((len(test_y), 1))
    inv_test_y = concatenate((test_y, test_X[:, -96:]), axis=1)
    test_y_inverse = scaler.inverse_transform(inv_test_y)
    test_y_inverse = test_y_inverse[:,0]

    y_pred = y_pred.reshape((len(y_pred), 1))
    inv_y_pred = concatenate((y_pred, test_X[:, -96:]), axis=1)
    y_pred_inverse = scaler.inverse_transform(inv_y_pred)
    y_pred_inverse = y_pred_inverse[:,0]

    # evaluate scaled errors
    mae = mean_absolute_error(test_y_inverse, y_pred_inverse)
    print('Inverted Mean Absolute Error:', mae)
    rmse = sqrt(mean_squared_error(test_y_inverse, y_pred_inverse))
    print('Inverted Test RMSE: %.3f' % rmse)
    r2 = r2_score(test_y_inverse, y_pred_inverse)
    print('Inverted Test R^2: %.3f' % r2)

(70080, 1, 95) (70080,) (17520, 1, 95) (17520,)
Train on 70080 samples, validate on 17520 samples
Epoch 1/50
70080/70080 [==============================] - 7s 102us/step - loss: 0.0267 - val_loss: 0.0084
Epoch 2/50
70080/70080 [==============================] - 6s 82us/step - loss: 0.0085 - val_loss: 0.0100
Epoch 3/50
70080/70080 [==============================] - 6s 84us/step - loss: 0.0073 - val_loss: 0.0063
Epoch 4/50
70080/70080 [==============================] - 6s 86us/step - loss: 0.0055 - val_loss: 0.0042
Epoch 5/50
70080/70080 [==============================] - 6s 88us/step - loss: 0.0049 - val_loss: 0.0039
Epoch 6/50
70080/70080 [==============================] - 6s 90us/step - loss: 0.0041 - val_loss: 0.0050
Epoch 7/50
70080/70080 [==============================] - 7s 104us/step - loss: 0.0038 - val_loss: 0.0034
Epoch 8/50
70080/70080 [==============================] - 8s 110us/step - loss: 0.0036 - val_loss: 0.0040
Epoch 9/50
70080/70080 [==============================] - 7

70080/70080 [==============================] - 7s 99us/step - loss: 0.0024 - val_loss: 0.0042
Epoch 26/50
70080/70080 [==============================] - 7s 98us/step - loss: 0.0024 - val_loss: 0.0011
Epoch 27/50
70080/70080 [==============================] - 7s 99us/step - loss: 0.0022 - val_loss: 0.0031
Epoch 28/50
70080/70080 [==============================] - 9s 125us/step - loss: 0.0020 - val_loss: 0.0016
Epoch 29/50
70080/70080 [==============================] - 10s 145us/step - loss: 0.0021 - val_loss: 0.0042
Epoch 30/50
70080/70080 [==============================] - 8s 112us/step - loss: 0.0020 - val_loss: 0.0026
Epoch 31/50
70080/70080 [==============================] - 8s 109us/step - loss: 0.0022 - val_loss: 8.8959e-04
Epoch 32/50
70080/70080 [==============================] - 8s 114us/step - loss: 0.0022 - val_loss: 0.0019
Epoch 33/50
70080/70080 [==============================] - 7s 107us/step - loss: 0.0019 - val_loss: 0.0013
Epoch 34/50
70080/70080 [======================

Epoch 50/50
70080/70080 [==============================] - 7s 98us/step - loss: 0.0013 - val_loss: 0.0019
Inverted Mean Absolute Error: 15.811977
Inverted Test RMSE: 17.195
Inverted Test R^2: 1.000


In [7]:
for node in flows:
    # split into train and test sets
    flow_values = node.values
    train, test = train_test_split(flow_values, test_size=0.2, random_state=0)

    # split into input and outputs
    train_X, train_y = train[:, :-1], train[:, -1]
    test_X, test_y = test[:, :-1], test[:, -1]

    # reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
    print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

    model = Sequential()
    model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam')
    # fit network
    model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=1, shuffle=False)
    
    # make predictions on the test data
    y_pred = model.predict(test_X)
    test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))

    test_y = test_y.reshape((len(test_y), 1))
    inv_test_y = concatenate((test_y, test_X[:, -96:]), axis=1)
    test_y_inverse = scaler.inverse_transform(inv_test_y)
    test_y_inverse = test_y_inverse[:,0]

    y_pred = y_pred.reshape((len(y_pred), 1))
    inv_y_pred = concatenate((y_pred, test_X[:, -96:]), axis=1)
    y_pred_inverse = scaler.inverse_transform(inv_y_pred)
    y_pred_inverse = y_pred_inverse[:,0]

    # evaluate scaled errors
    mae = mean_absolute_error(test_y_inverse, y_pred_inverse)
    print('Inverted Mean Absolute Error:', mae)
    rmse = sqrt(mean_squared_error(test_y_inverse, y_pred_inverse))
    print('Inverted Test RMSE: %.3f' % rmse)
    r2 = r2_score(test_y_inverse, y_pred_inverse)
    print('Inverted Test R^2: %.3f' % r2)

(70080, 1, 95) (70080,) (17520, 1, 95) (17520,)
Train on 70080 samples, validate on 17520 samples
Epoch 1/50
70080/70080 [==============================] - 8s 113us/step - loss: 0.0080 - val_loss: 0.0059
Epoch 2/50
70080/70080 [==============================] - 6s 84us/step - loss: 0.0038 - val_loss: 0.0018
Epoch 3/50
70080/70080 [==============================] - 6s 79us/step - loss: 0.0032 - val_loss: 9.6646e-04
Epoch 4/50
70080/70080 [==============================] - 5s 78us/step - loss: 0.0030 - val_loss: 0.0072
Epoch 5/50
70080/70080 [==============================] - 5s 78us/step - loss: 0.0031 - val_loss: 0.0014
Epoch 6/50
70080/70080 [==============================] - 5s 78us/step - loss: 0.0028 - val_loss: 9.0210e-04
Epoch 7/50
70080/70080 [==============================] - 6s 83us/step - loss: 0.0023 - val_loss: 0.0040
Epoch 8/50
70080/70080 [==============================] - 6s 91us/step - loss: 0.0025 - val_loss: 0.0040
Epoch 9/50
70080/70080 [=============================

70080/70080 [==============================] - 5s 78us/step - loss: 0.0032 - val_loss: 0.0021
Epoch 25/50
70080/70080 [==============================] - 6s 87us/step - loss: 0.0032 - val_loss: 0.0023
Epoch 26/50
70080/70080 [==============================] - 6s 86us/step - loss: 0.0030 - val_loss: 0.0022
Epoch 27/50
70080/70080 [==============================] - 6s 89us/step - loss: 0.0032 - val_loss: 0.0028
Epoch 28/50
70080/70080 [==============================] - 6s 81us/step - loss: 0.0030 - val_loss: 0.0026
Epoch 29/50
70080/70080 [==============================] - 6s 84us/step - loss: 0.0030 - val_loss: 0.0023
Epoch 30/50
70080/70080 [==============================] - 6s 86us/step - loss: 0.0030 - val_loss: 0.0050
Epoch 31/50
70080/70080 [==============================] - 6s 87us/step - loss: 0.0031 - val_loss: 0.0022
Epoch 32/50
70080/70080 [==============================] - 6s 87us/step - loss: 0.0030 - val_loss: 0.0030
Epoch 33/50
70080/70080 [==============================] -

70080/70080 [==============================] - 6s 90us/step - loss: 0.0083 - val_loss: 0.0074
Epoch 50/50
70080/70080 [==============================] - 6s 85us/step - loss: 0.0083 - val_loss: 0.0071
Inverted Mean Absolute Error: 60.795097
Inverted Test RMSE: 81.062
Inverted Test R^2: 0.997
